In [ ]:
import pandas as pd
import glob

In [ ]:
files = glob.glob('sensordata/*/combined.csv') # Get list of files. This is created in pre_processing.ipynb

In [ ]:
lowest_timestamp_interval = #FILL IN HERE

# Merges all the X (where X is the number of sensors) number of combined csv files into a single one
# where each combined file is a column in the new csv file. Meaning you get a new csv file with
# 1 (timestamp) + X - number of columns.
# Windows are created which sensor recordings are added. Say that your lowest_timestamp_interval is 30 minutes.
# You would then have a window for for example 16:00 and 16:30. Now any sensor recording, recorded between
# 16:00 and 16:30 would be placed in the 16:00 window, while anything between 16:30 and 17:00 would be placed in
# the 16:30 window, and so on.


def merge_combined_files(start_date, end_date):
    # A date range from before the first  is used to be able to merge the data in given windows.
    # For this range, the lowest timestamp interval is used, in order to be able to merge all the data.
    date_range = pd.date_range(start=start_date, end=end_date, freq=lowest_timestamp_interval)
    df_range = pd.DataFrame(date_range, columns=['timestamp'])
    for i, file in enumerate(files):
        filename = file.split('/')[1]
        print(i+1, filename)
        df = pd.read_csv(file, usecols=['timestamp', filename], sep=";")
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        if (i < 1):
            new_df = df_range
        else:
            new_df = pd.merge_asof(new_df, df, on='timestamp', direction='forward',
                                   tolerance=pd.Timedelta(lowest_timestamp_interval))
    
    new_df.set_index('timestamp', inplace=True)
    new_df.dropna(how='all', inplace=True)    
    #new_df.to_csv('merge_window.csv', sep=";")
    return new_df

In [ ]:
# Interpolates data, in order to have enough rows for training the data

def interpolate(df, technique, max_na, columns_to_interpolate, new_filename='new_file', write_to_csv=False):
    columns = ['timestamp'] + columns_to_interpolate
    interpolated_df = df[columns].interpolate(method=technique, axis=0, limit=max_na, limit_direction='both')
    print('-----------------------------------------------------------------')
    print('-----------------------------------------------------------------')
    print(interpolated_df)
    print('-----------------------------------------------------------------')
    print('-----------------------------------------------------------------')
    df.update(interpolated_df, overwrite=False)
    if (write_to_csv):
        df.to_csv(new_filename, sep=";")
        return
    return df

In [ ]:
# This removes negative values in the dataset. Certain cells had value of -999, which does not make any sense

df_merged = pd.read_csv('merge_window.csv', index_col=0, sep=";")
df_removed_noise = df_merged.copy()

for col in df_merged.columns.values[1:]:
    df_removed_noise = df_removed_noise[~(df_removed_noise[col] < 0)]

# df_removed_noise.to_csv('merge_removed_noise.csv', sep=";")

In [ ]:
list_interval_A = [] # Enter the items of a given interval. A list (or optionally multidimensional lists)
                     # should be used for each different interval, because they will be interpolated differently

list_interval_B = [] # This is because 2 steps for an interval of an hour, is different to 2 steps, for 1 min interval

interval_A = # Some interval, that the items in list_interval_A have
interval_B = # Some other interval

# Below lists of the different intervals are created. This can be done in a much more clever way,
# but was sufficient for the few amount of intervals in this project.

df_interpolated = interpolate(df_removed_noise, 'linear', interval_A, list_interval_A)
df_interpolated = interpolate(df_removed_noise, 'linear', interval_B, list_interval_B)

# df_interpolated_noise.to_csv('merge_removed_noise.csv', sep=";")